<a href="https://colab.research.google.com/github/tc3oliver/LangChain-Guide/blob/dev/05_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/LangChain')
with open('env_vars.txt', 'r') as file:
  for line in file:
    key, value = line.strip().split('=')
    os.environ[key] = value

!pip install langchain==0.0.247 openai

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00


In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [3]:
conversation({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi', additional_kwargs={}, example=False),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, example=False)],
 'text': 'Hello! How can I assist you today?'}

In [4]:
conversation.run({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How can I assist you today?
Human: hi

> Finished chain.


'Hello again! How can I help you today?'

In [10]:
nickbot = LLMChain(
    llm=llm,
    prompt=ChatPromptTemplate(
      messages=[
          MessagesPlaceholder(variable_name="nick"),
          HumanMessagePromptTemplate.from_template("{question}")
      ]),
    verbose=True,
    memory=ConversationBufferMemory(memory_key="nick", return_messages=True)
)

In [13]:
bobbot = LLMChain(
    llm=llm,
    prompt=ChatPromptTemplate(
      messages=[
          MessagesPlaceholder(variable_name="bob"),
          HumanMessagePromptTemplate.from_template("{question}")
      ]),
    verbose=True,
    memory=ConversationBufferMemory(memory_key="bob", return_messages=True)
)

In [11]:
nickbot.run({"question": "hi I'm Nick"})



> Entering new LLMChain chain...
Prompt after formatting:
Human: hi I'm Nick

> Finished chain.


'Hello Nick! How can I assist you today?'

In [14]:
bobbot.run({"question": "hi I'm Bob"})



> Entering new LLMChain chain...
Prompt after formatting:
Human: hi I'm Bob

> Finished chain.


'Hello Bob! How can I assist you today?'

In [16]:
nickbot.run({"question": "Do you know my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
Human: hi I'm Nick
AI: Hello Nick! How can I assist you today?
Human: Do you know who I am?
AI: As an AI language model, I don't have access to personal information unless you provide it to me. Could you please tell me more about yourself?
Human: Do you know my name?

> Finished chain.


'Yes, you mentioned that your name is Nick. How can I assist you, Nick?'

In [18]:
def userBot(name):
  return LLMChain(
    llm=llm,
    prompt=ChatPromptTemplate(
      messages=[
          MessagesPlaceholder(variable_name=name),
          HumanMessagePromptTemplate.from_template("{question}")
      ]),
    verbose=True,
    memory=ConversationBufferMemory(memory_key=name, return_messages=True)
  )

In [20]:
userBot("Oliver").run({"question": "你好!我是 Oliver"})



> Entering new LLMChain chain...
Prompt after formatting:
Human: 你好!我是 Oliver

> Finished chain.


'你好 Oliver! 有什么我可以帮助你的吗？'

In [21]:
userBot("Oliver").run({"question": "我的名字是什麼"})



> Entering new LLMChain chain...
Prompt after formatting:
Human: 我的名字是什麼

> Finished chain.


'很抱歉，我無法得知您的真實姓名。'